In [1]:
# https://www.kaggle.com/ngbolin/mnist-dataset-digit-recognizer/data?select=sample_submission.csv
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

import tensorflow as tf

In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [3]:
df_train.head() # 784 features, 1 label

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [5]:
df_test.head()
df_label

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 1212)

X_train = X_train.to_numpy().reshape(33600, 784) #(33600, 784)
X_cv = X_cv.to_numpy().reshape(8400, 784) #(8400, 784)

X_test = X_test.to_numpy().reshape(28000, 784)

In [7]:
print((min(X_train[1]), max(X_train[1])))

(0, 255)


In [8]:
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [9]:
shared_percent = 75 / 100
shared_train_data_size = int(X_train.shape[0] * shared_percent)
num_models = 2

shared_train_data = X_train[:shared_train_data_size]
shared_train_labels = y_train[:shared_train_data_size]

private_train_data = np.split(X_train[shared_train_data_size:], num_models)
private_train_labels = np.split(
    y_train[shared_train_data_size:], num_models)

print(X_train.shape)
print(shared_train_data.shape)
print(private_train_data[1].shape)

(33600, 784)
(25200, 784)
(4200, 784)


In [10]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

learning_rate = 0.1
training_epochs = 20
batch_size = 100

In [11]:
def get_model_ins():
    Inp = Input(shape=(784,))
    x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
    x = Dropout(0.3)(x)
    x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
    x = Dropout(0.3)(x)
    x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
    x = Dropout(0.3)(x)
    x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
    output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)
    
    model = Model(Inp, output)
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

def save_weights(model, prefix):
    for i, layer in enumerate(model.layers):
        if len(layer.get_weights()) == 0:
            continue
        m = layer.get_weights()
        np.savetxt('saved_weights/{}-w{}.out'.format(prefix, i), m[0], header="{},{}".format(*m[0].shape), delimiter=",")
        np.savetxt('saved_weights/{}-b{}.out'.format(prefix, i), m[1], header="{},{}".format(m[1].shape, 1), delimiter=",")

In [12]:
main_model = get_model_ins()
main_model.summary()
og_file_name = 'saved_model/main_model_{}_{}.h5'.format(int(shared_percent * 100), num_models)
main_model.save(og_file_name)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)              

In [13]:
for i in range(num_models):
    print("Training model - {}".format(i))
    
    model_train_data = np.vstack((shared_train_data, private_train_data[i]))
    model_train_labels = np.vstack(
        (shared_train_labels, private_train_labels[i]))

    model = keras.models.load_model(og_file_name)
    
#     model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
    
    model.summary()
    
    history = model.fit(model_train_data, model_train_labels, batch_size = batch_size, epochs = training_epochs, validation_data=(X_cv, y_cv))
    
    test_pred = pd.DataFrame(model.predict(X_test, batch_size=200))
    test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
    test_pred.index.name = 'ImageId'
    test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
    test_pred['ImageId'] = test_pred['ImageId'] + 1

    test_pred.head()
    
    model_name = "saved_model/model_{}_{}_{}.h5".format(i, int(shared_percent * 100), num_models)
    model.save(model_name)
    
    save_weights(model, 'model_{}_{}_{}'.format(i, int(shared_percent * 100), num_models))

Training model - 0
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None,

In [14]:
import time

def test():
    model = keras.models.load_model("saved_big/saved_model/modela.h5")

s = time.perf_counter()
test()
e = time.perf_counter()
print(e-s)

2.559471486999996


In [15]:
model = keras.models.load_model("saved_model/model_0_25_2.h5")

In [52]:
import math

def split(array, nrows, ncols):
    """Split a matrix into sub-matrices."""

    r, h = array.shape
    if r % nrows != 0:
        padding = (math.ceil(r / nrows) * nrows) - r
        array = np.vstack((array, np.zeros((padding, h))))
        r, h = array.shape
    if h % ncols != 0:
        padding = (math.ceil(h / ncols) * ncols) - h
        array = np.hstack((array, np.zeros((r, padding))))
        r, h = array.shape

    print(array.shape)
        
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))

j = np.array([
    [1,2,3,4],
    [5,6,7,8],
    [9,10,11,12],
    [13,14,15,16]
])

def gather_blocks_to_pages(blocks, num_elem_per_page):
    pages = []
    i = 0

    while i < len(blocks):
        count = 0
        page = []
        while i < len(blocks) and count + blocks[i].shape[0] * blocks[i].shape[1] <= num_elem_per_page:
            page.append(blocks[i])
            count += blocks[i].shape[0] * blocks[i].shape[1]
            i += 1
        pages.append(page)
        print("Adding {} elems to page {}".format(count, len(pages) - 1))
    return pages
            
print(model.layers[1].get_weights()[0].shape)
x = split(model.layers[1].get_weights()[0], 5, 5)
ps = gather_blocks_to_pages(x, 63725) # ~1MB 16 bytes * 63725

(784, 300)
(785, 300)
Adding 63725 elems to page 0
Adding 63725 elems to page 1
Adding 63725 elems to page 2
Adding 44325 elems to page 3


In [50]:
len(ps)

4